In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
using NonlinearSolve
import DifferentialEquations as DE
include("multiharmonic_balance.jl");

Equation parameters

In [ ]:
g = 9.81;
height = 3.0;
omega = 10.0;

Discretization parameters

In [ ]:
harmonics = 1; # number of harmonics
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
Nx = 40;
stepx = (xright-xleft)/Nx;
u0 = zeros((Nx+1) * harmonics * 4);

**Declare the problem**

In [ ]:
# Define symbolics
Model.@parameters x, t;

const Dx = Model.Differential(x);
const Dt = Model.Differential(t);

In [ ]:
vars, var_exprs, fields = create_ansatz((x,), t, omega, harmonics, 2);
u = fields[1];
eta = fields[2];
pdes = [Dt(eta) + height * Dx(u),
       Dt(u) + g * Dx(eta)];

In [ ]:
u

In [ ]:
eta

In [ ]:
vars

In [ ]:
function simplify_problem(pdes, t, omega, harmonics, Nx, vars)
    all_sym_eqs = Expr[]
    
    for pde in pdes
        expanded = expand_trig_jl(pde, t, omega)
        eqns = make_residual(expanded, harmonics, omega, t)
        sym_eqs = map(transform_sym(Nx) ∘ Meta.parse ∘ string, eqns)
        append!(all_sym_eqs, sym_eqs)
    end
    resid = create_residual_function(all_sym_eqs, vars, Nx, 0,
    Dict(
    :A => [:(exp(-x^2)), nothing],
    :B => [:(0), nothing],
    :C => [nothing, :(0)],
    :D => [nothing, :(0)],
    )
)
    return resid
end

function solve_problem(resid, harmonics, Nx, u0, stepx; Ny=0, stepy=0.0)
    residual! = eval(resid)
    
    is_2D = Ny > 0
    N = is_2D ? (Nx+1) * (Ny+1) : (Nx+1)
    p = is_2D ? [stepx, stepy] : [stepx]
    
    R = similar(u0)
    
    jac_sparsity = Symbolics.jacobian_sparsity(
        (R, U) -> Base.invokelatest(residual!, R, U, p), 
        R, u0
    )
    
    wrapped_residual! = (R, U, p) -> Base.invokelatest(residual!, R, U, p)
    
    f = NonlinearFunction(wrapped_residual!; jac_prototype=float.(jac_sparsity))
    prob = NonlinearProblem(f, u0, p)
    sol = solve(prob, NewtonRaphson(), reltol=1e-5, abstol=1e-5, maxiters=1000)
    
    println(sol.retcode)
    
    if is_2D
        solutions = [reshape(sol.u[(k-1)*N+1:k*N], Nx+1, Ny+1) for k in 1:(2*harmonics)]
    else
        solutions = [sol.u[(k-1)*N+1:k*N] for k in 1:(2*harmonics)]
    end
    
    return solutions, sol
end

In [ ]:
compute_time = @elapsed begin
    resid = simplify_problem(pdes, t, omega, harmonics, Nx, vars);
    solutions, sol = solve_problem(resid, harmonics, Nx, u0, stepx)
end;
println("Compute time: $compute_time seconds")

In [ ]:
@time begin

end

In [ ]:
Dict(:A => [exp(-x^2), 0], :C => [0, 0])